In [1]:
import numpy as np
import os, sys
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from collections import Counter
from tqdm.auto import tqdm, trange
from sklearn.preprocessing import MinMaxScaler
import re
import concurrent.futures
from sklearn.metrics import silhouette_score
from scipy.stats import pearsonr
import copy
import pickle
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, DistributedSampler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from kmeans_pytorch import kmeans

from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import CTGANSynthesizer
from sdv.sampling import Condition
from sdv.evaluation.single_table import get_column_plot

import dask.dataframe as dpd
import dask_geopandas as dgpd
from dask.diagnostics import ProgressBar
from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

np.random.seed(0)

/home/grads/l/lipai.huang/GAN_simulated_events/se_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = Client(n_workers=80) #192 totally

In [50]:
client.close()

In [3]:
print(torch.__version__, torch.cuda.is_available())
torch.cuda.set_device(0)

2.0.1+cu118 True


In [4]:
if torch.cuda.device_count() >= 1:
    print(f"We have {torch.cuda.device_count()} GPUs!")

We have 1 GPUs!


In [5]:
# Plot geo map
def plot_map(gdf, col, vmin=0, vmax=300, figsize=(8, 6), dpi=200, notes='', to_path='', dots=[], title=True, s=1):
    plt.clf()
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # Plot without specifying legend_kwds
    try:
        gdf.plot(ax=ax, column=col, cmap='coolwarm', vmin=vmin, vmax=vmax, cax=cax, s=s)
    except:
        gdf.plot(ax=ax, column=col, cmap='coolwarm', vmin=vmin, vmax=vmax, cax=cax)
    if dots:
        for dot in dots:
            gdf.loc[dot:dot].plot(ax=ax, linewidth=1, color='black', alpha=0.5)
            ax.text(gdf.loc[dot, 'x'], gdf.loc[dot, 'y'], str(dot), fontsize=12)

    # Create colorbar with custom font size
    sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cax, 
#                         label=col.upper(), 
                        shrink=.5)
#     cbar.ax.tick_params(labelsize=20)  # Set the font size for the colorbar
#     bd_gdf.boundary.plot(ax=ax, linewidth=1, color='k')
    # Change tick fontsize
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.set_xticks([])
    ax.set_yticks([])
#     ax.scatter(gdf['x'], gdf['y'], s=1, c='k')
    
    # Change color bar fontsize
#     cbar.set_label(col.upper(), fontsize=20)
    if title:
        ax.set_title(f'{col.upper()}')
#     if not os.path.exists(f'plots/test/{notes}'):
#         os.mkdir(f'plots/test/{notes}')
#     fig.savefig(f'plots/test/{notes}/{col}.png')
    if to_path:
        fig.savefig(f'{to_path}')

In [5]:
coord_gdf = gpd.read_file('../src/coord/coord_gdf.shp')
coord_gdf = coord_gdf.drop(columns=['cell_rmse1', 'cell_r21', 'cell_rmse2', 'cell_r22', 'depth'])
coord_gdf

,x,y,ter,HUC12,region,channel,geometry
0,2.933766e+06,1.396557e+07,301.388702,Cypress Creek,0,0,"POLYGON ((2934366.000 13964974.635, 2933003.17..."
1,2.934966e+06,1.396557e+07,301.594696,Cypress Creek,0,0,"POLYGON ((2934366.000 13967369.160, 2934380.33..."
2,2.933766e+06,1.396437e+07,294.629181,Cypress Creek,0,0,"POLYGON ((2934366.000 13964974.635, 2934366.00..."
3,2.934966e+06,1.396437e+07,298.529877,Cypress Creek,0,0,"POLYGON ((2935566.000 13963774.635, 2934366.00..."
4,2.936166e+06,1.396437e+07,294.815002,Cypress Creek,0,0,"POLYGON ((2936766.000 13963774.635, 2935566.00..."
...,...,...,...,...,...,...,...
26296,3.039069e+06,1.385008e+07,54.643570,Whiteoak Bayou-Buffalo Bayou,2,1,"POLYGON ((3039427.707 13849492.726, 3038745.86..."
26297,3.039053e+06,1.385088e+07,59.625050,Addicks Reservoir,3,1,"POLYGON ((3039399.212 13851153.541, 3039405.50..."
26298,3.038396e+06,1.385006e+07,60.055576,Whiteoak Bayou-Buffalo Bayou,2,0,"POLYGON ((3038723.769 13850469.724, 3038724.68..."
26299,3.038392e+06,1.385087e+07,59.625050,Addicks Reservoir,3,0,"POLYGON ((3038721.900 13851266.014, 3038723.76..."


In [6]:
watershed_dict = coord_gdf.drop_duplicates(subset='HUC12').set_index('HUC12')['region'].to_dict()
watershed_dict

{'Cypress Creek': 0,
 'Greens Bayou': 1,
 'Whiteoak Bayou-Buffalo Bayou': 2,
 'Addicks Reservoir': 3,
 'Barker Reservoir': 4,
 'Hunting Bayou': 5,
 'Vince Bayou-Buffalo Bayou': 6,
 'Brays Bayou': 7,
 'Sims Bayou': 8}

In [7]:
scaler = MinMaxScaler()
xy_scaled = scaler.fit_transform(coord_gdf[['x', 'y']])

def load_and_scale(file_path, scale=False):
    df = pd.read_parquet(file_path)[['x', 'y', 'channel', 'ter', 'cumu_rain', 'peak_int', 'duration', 'depth']]
    if scale:
        df[['x', 'y']] = xy_scaled
    return df

file_paths = [f'../src/tables/data{i}.parquet' for i in range(1, len([f for f in os.listdir('../src/tables') if f.endswith('.parquet')]) + 1)]
events = [load_and_scale(file) for file in file_paths]
with ProgressBar():
    result = dpd.concat(events, axis=0)
events_df = result.compute()

In [8]:
events_df

,x,y,channel,ter,cumu_rain,peak_int,duration,depth
0,2.933766e+06,1.396557e+07,0,301.388702,3.001601,3.001601,1,3.866364
1,2.934966e+06,1.396557e+07,0,301.594696,3.127318,3.127318,1,2.150513
2,2.933766e+06,1.396437e+07,0,294.629181,3.211096,3.211096,1,3.595856
3,2.934966e+06,1.396437e+07,0,298.529877,3.260372,3.260372,1,2.782227
4,2.936166e+06,1.396437e+07,0,294.815002,3.309647,3.309647,1,2.787598
...,...,...,...,...,...,...,...,...
26296,3.039069e+06,1.385008e+07,1,54.643570,0.000000,0.000000,2,0.000000
26297,3.039053e+06,1.385088e+07,1,59.625050,0.000000,0.000000,2,0.000000
26298,3.038396e+06,1.385006e+07,0,60.055576,0.000000,0.000000,2,0.000000
26299,3.038392e+06,1.385087e+07,0,59.625050,0.000000,0.000000,2,0.000000


In [9]:
feature_cols = ['channel', 'ter', 'cumu_rain', 'peak_int', 'duration']
features = events_df[feature_cols].values
targets = events_df[['depth']].values

X_train, X_temp, y_train, y_temp = train_test_split(
    features, targets, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [10]:
dtrain = xgb.DMatrix(X_train_scaled, y_train)
dval = xgb.DMatrix(X_val_scaled, y_val)
dtest = xgb.DMatrix(X_test_scaled, y_test)

In [11]:
def loss_metric_eval(y_pred, dtrue, loss_str):
    y_pred = np.maximum(0, y_pred)
    loss_str = loss_str.lower()
    y_true = dtrue.get_label()
    if loss_str == "rmsle":
        return loss_str, np.sqrt(np.mean((np.log(y_true+1) - np.log(y_pred+1))**2))
    elif loss_str == "mae":
        return loss_str, np.mean(np.abs(y_true - y_pred))
    else:
        return 'rmse', np.sqrt(np.mean((y_true - y_pred)**2))

In [12]:
params = {
    'tree_method': 'gpu_hist',
    'subsample': 0.9,
    'n_estimators': 1000,
    'min_child_weight': 2,
    'max_depth': 8,
    'learning_rate': 0.005,
    'gamma': 0.1,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.85,
    "objective": "reg:squarederror",
    "max_bin": 2048,
    "eval_metric": "rmse",
    "gpu_id": 0,
    "alpha": 1,
    "lambda": 10
}

In [13]:
best_checkpoint = os.path.join('../checkpoints/depth', f'XGBoost.mod')
train_history = []
val_history = []
test_history_100 = []

In [14]:
def loss_metric_eval(y_pred, dtrue, loss_str):
    y_pred = np.maximum(0, y_pred)
    loss_str = loss_str.lower()
    y_true = dtrue.get_label()
    if loss_str == "rmsle":
        return loss_str, np.sqrt(np.mean((np.log(y_true+1) - np.log(y_pred+1))**2))
    elif loss_str == "mae":
        return loss_str, np.mean(np.abs(y_true - y_pred))
    else:
        return 'rmse', np.sqrt(np.mean((y_true - y_pred)**2))

In [15]:
class CustomEvaluationCallback(xgb.callback.TrainingCallback):
    def __init__(self, dtest, eval_every=100):
        self.dtest = dtest
        self.eval_every = eval_every
        self.best_loss = float("inf")
        self.best_epoch = 0

    def after_iteration(self, model, epoch, evals_log):
        global best_epoch
        # Record the training RMSE
        train_loss = evals_log['train']['rmse'][-1]
        train_history.append(train_loss)

        # Evaluate on the validation set
        val_loss = evals_log['val']['rmse'][-1]
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            model.save_model(best_checkpoint)
            self.best_epoch = epoch
#             print(f'Epoch {epoch} {loss_metric}={val_loss:.3f} saved better checkpoint to {best_checkpoint}')
        val_history.append(val_loss)

        if epoch % self.eval_every == 0 and epoch > 0:
            test_preds = np.maximum(0, model.predict(self.dtest))
            _, test_loss = loss_metric_eval(test_preds, self.dtest, 'rmse')
            test_history_100.append(test_loss)
        return False

In [16]:
custom_eval_cb = CustomEvaluationCallback(dtest)
feval_wrapper = lambda y_pred, dtrue: loss_metric_eval(y_pred, dtrue, 'rmse')

In [29]:
model = xgb.train(
        params,
        dtrain,
        num_boost_round=5000,
        evals=[(dtrain, "train"), (dval, "val")],
        feval=feval_wrapper,
        callbacks=[custom_eval_cb],
    )

[0]	train-rmse:4.51968	val-rmse:4.51129
[1]	train-rmse:4.51021	val-rmse:4.50183
[2]	train-rmse:4.49842	val-rmse:4.49003
[3]	train-rmse:4.48908	val-rmse:4.48069
[4]	train-rmse:4.47936	val-rmse:4.47098
[5]	train-rmse:4.47631	val-rmse:4.46793
[6]	train-rmse:4.46794	val-rmse:4.45956
[7]	train-rmse:4.45708	val-rmse:4.44870
[8]	train-rmse:4.44746	val-rmse:4.43909
[9]	train-rmse:4.44285	val-rmse:4.43450
[10]	train-rmse:4.43614	val-rmse:4.42779
[11]	train-rmse:4.42490	val-rmse:4.41655
[12]	train-rmse:4.42063	val-rmse:4.41227
[13]	train-rmse:4.41601	val-rmse:4.40766
[14]	train-rmse:4.40700	val-rmse:4.39866
[15]	train-rmse:4.39806	val-rmse:4.38973
[16]	train-rmse:4.38920	val-rmse:4.38087
[17]	train-rmse:4.38040	val-rmse:4.37208
[18]	train-rmse:4.37122	val-rmse:4.36292
[19]	train-rmse:4.36213	val-rmse:4.35383
[20]	train-rmse:4.35579	val-rmse:4.34749
[21]	train-rmse:4.35124	val-rmse:4.34293
[22]	train-rmse:4.34847	val-rmse:4.34016
[23]	train-rmse:4.34514	val-rmse:4.33683
[24]	train-rmse:4.33486	va

In [30]:
test_history_100

[3.9205925,
 3.543781,
 3.2840247,
 3.1362164,
 3.0266807,
 2.9566548,
 2.910352,
 2.872995,
 2.8429496,
 2.8229938,
 2.806383,
 2.7953117,
 2.7834256,
 2.7735963,
 2.7647772,
 2.759687,
 2.7547073,
 2.7502043,
 2.745644,
 2.7418473,
 2.7380636,
 2.7345946,
 2.731373,
 2.728231,
 2.7263124,
 2.7242079,
 2.7210448,
 3.9205925,
 3.543781,
 3.2840247,
 3.1362164,
 3.0266807,
 2.9566548,
 2.910352,
 2.872995,
 2.8429496,
 2.8229938,
 2.806383,
 2.7953117,
 2.7834256,
 2.7735963,
 2.7647772,
 2.759687,
 2.7547073,
 2.7502043,
 2.745644,
 2.7418473,
 2.7380636,
 2.7345946,
 2.731373,
 2.728231,
 2.7263124,
 2.7242079,
 2.7210448,
 2.7191129,
 2.7174656,
 2.7149806,
 2.713006,
 2.7110765,
 2.7091231,
 2.7076516,
 2.7063875,
 2.70476,
 2.7032957,
 2.7023854,
 2.701254,
 2.7002163,
 2.6991103,
 2.6982486,
 2.6974785,
 2.696477,
 2.6956975,
 2.6949248,
 2.6942248,
 2.6933646,
 2.6926272]

In [17]:
params = {
    **params,
    'n_jobs': 80
}

In [18]:
model = xgb.XGBRegressor(**params)
model.load_model(best_checkpoint)

In [26]:
y_preds = np.maximum(0, model.predict(X_test_scaled))
y_trues = y_test.reshape(-1,)

In [45]:
def calc_rmse(y_trues, y_preds, indices=[]):
    if len(indices) > 0:
        y_trues = y_trues[indices]
        y_preds = y_preds[indices]
    return np.round(np.sqrt(np.mean((y_preds[:] - y_trues[:]) ** 2)), 4)

def calc_r2(y_trues, y_preds, indices=[]):
    if len(indices) > 0:
        y_trues = y_trues[indices]
        y_preds = y_preds[indices]
    return np.round(1 - (np.sum((y_preds - y_trues)**2))/(np.sum((y_trues - np.mean(y_trues))**2)), 4)

In [46]:
test_rmse = calc_rmse(y_trues, y_preds)
test_r2 = calc_r2(y_trues, y_preds)
test_result_dict = {
    'rmse': test_rmse,
    'r2': test_r2,
}

In [47]:
channel_indices = np.where(scaler.inverse_transform(X_test_scaled)[:, 0] == 1)[0]
non_channel_indices = np.where(scaler.inverse_transform(X_test_scaled)[:, 0] == 0)[0]

In [48]:
channel_rmse = calc_rmse(y_trues, y_preds, channel_indices)
channel_r2 = calc_r2(y_trues, y_preds, channel_indices)
non_channel_rmse = calc_rmse(y_trues, y_preds, non_channel_indices)
non_channel_r2 = calc_r2(y_trues, y_preds, non_channel_indices)
test_result_dict['channel_rmse'] = channel_rmse
test_result_dict['channel_r2'] = channel_r2
test_result_dict['non_channel_rmse'] = non_channel_rmse
test_result_dict['non_channel_r2'] = non_channel_r2

In [49]:
test_result_dict

{'rmse': 2.6916,
 'r2': 0.6463,
 'channel_rmse': 4.2755,
 'channel_r2': 0.7078,
 'non_channel_rmse': 2.4153,
 'non_channel_r2': 0.492}